
# **Analysis of Covid-19 in Germany**

In this project we present the evolution of cases from day zero in Germany, as well as deaths, the number of free and occupied beds by day.


At the beginning, we imported data from trusted locations and started the cleaning process to avoid duplicate data and those that are not necessary for this study.





In [ ]:
# Imports
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


Mounted at /content/drive


In [ ]:
#Read hospitals file for all bundesländer
nrw = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/NRW.csv")
nrw.insert(0,"Bundesland","Nordrhein-Westfalen")
bw = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/badenWuerttemberg.csv")
bw.insert(0,"Bundesland","Baden-Würtemberg")
bayern = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/bayern.csv")
bayern.insert(0,"Bundesland","Bayern")
berlin = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/berlin.csv")
berlin.insert(0,"Bundesland","Berlin")
brandenburg = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/brandenburg.csv")
brandenburg.insert(0,"Bundesland","Brandenburg")
bremen = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/bremen.csv")
bremen.insert(0,"Bundesland","Bremen")
hamburg = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/hamburg.csv")
hamburg.insert(0,"Bundesland","Hamburg")
hessen = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/hessen.csv")
hessen.insert(0,"Bundesland","Hessen")
mvp = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/meckpom.csv")
mvp.insert(0,"Bundesland","Mecklenburg-Vorpommern")
ns = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/niedersachsen.csv")
ns.insert(0,"Bundesland","Niedersachsen")
rp = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/rheinlandPfalz.csv")
rp.insert(0,"Bundesland","Rheinland Pfalz")
saarland = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/saarland.csv")
saarland.insert(0,"Bundesland","Saarland")
sachsen = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/sachsen.csv")
sachsen.insert(0,"Bundesland","Sachsen")
sachsenanhalt = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/sachsenAnhalt.csv")
sachsenanhalt.insert(0,"Bundesland","Sachsen-Anhalt")
sh = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/schleswigHolstein.csv")
sh.insert(0,"Bundesland","Schleswig-Holstein")
thueringen = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/thueringen.csv")
thueringen.insert(0,"Bundesland","Thüringen")

In [ ]:
# Merge into one dataframe "hospitalsmerged"
hospitalsMerged =pd.concat([nrw,bw,bayern,berlin,brandenburg,bremen,hamburg,hessen,mvp,ns,rp,saarland,sachsen,sachsenanhalt,sh,thueringen])

In [ ]:
# Change "meldedatum" from object to datetime
hospitalsMerged.dtypes
for i in range(len(hospitalsMerged)) :
  date = str(hospitalsMerged.iloc[i, 1]).split("T")
  hospitalsMerged.iloc[i, 1]= date[0]

hospitalsMerged["date"]= pd.to_datetime(hospitalsMerged["date"],format='%Y-%m-%d')

In [ ]:
# Read corona cases and deaths for all German "kreise"
casesAndDeaths = pd.read_csv("/content/drive/My Drive/TechlabsProjectGroup19/RKI_COVID19.csv")
#Drop redundant columns
cols = [0,1,3,9,10,13,14,15,16,17]
casesAndDeaths=casesAndDeaths.drop(casesAndDeaths.columns[cols], axis=1)
casesAndDeaths.head(50)

# 2º Step

In this phase, we perform the cleaning of the data and its preparation, for possible manipulation and creation of diagrams without the occurrence of concordance errors.

*Converting the data type to the correct one (example: data that indicates dates) is essential to avoid tracebacks throughout the creation of the script.*

In [ ]:
# Change "meldedatum" from object to datetime
casesAndDeaths.dtypes
casesAndDeaths["Meldedatum"]= pd.to_datetime(casesAndDeaths["Meldedatum"],format='%Y/%m/%d %H:%M:%S')
casesAndDeaths.dtypes

In [ ]:
#sort cases death and sex by bundesland and date 
casesAndDeaths.sort_values(by=['Bundesland', 'Meldedatum'], inplace=True)
casesAndDeaths.head(50)

In [ ]:
# Get cases and deaths for each bundesland on each day
bundeslaender = ["Schleswig-Holstein","Hamburg","Niedersachsen","Bremen","Nordrhein-Westfalen","Hessen","Rheinland-Pfalz","Baden-Württemberg","Bayern","Saarland","Berlin","Brandenburg","Mecklenburg-Vorpommern","Sachsen","Sachsen-Anhalt","Thüringen"]
yearindex = pd.date_range(start="2020-01-01",end="2020-12-31",freq='D')
merged = pd.DataFrame(index=yearindex)
for bundesland in bundeslaender:
  casesAndDeathsBundesland = casesAndDeaths[casesAndDeaths['Bundesland']== bundesland]
  redundantcolumns = [1,6,7]
  casesAndDeathsBundesland = casesAndDeathsBundesland.drop(casesAndDeathsBundesland.columns[redundantcolumns], axis=1)
  casesAndDeathsBundesland=casesAndDeathsBundesland[['Bundesland','AnzahlFall', 'AnzahlTodesfall', 'Meldedatum']].groupby('Meldedatum').sum()
  casesAndDeathsBundesland.insert(0,bundesland,bundesland)

In [ ]:
# Get free and available beds + emergency reserve and deaths for each bundesland on each day

holder = hospitalsMerged[hospitalsMerged['Bundesland'] == bundesland]
holder = holder.set_index("date")
holder = holder.drop(columns=["Bundesland"])

In [ ]:
# Merge cases, deaths, available and free beds + emergency capacity 
casesAndDeathsBundesland = pd.concat([casesAndDeathsBundesland,holder],axis=1)

if (len(merged)==0):
  merged = casesAndDeathsBundesland
else:
  merged = pd.concat([merged,casesAndDeathsBundesland],axis=1)

In [ ]:
#Fill empty cells with 0/name of bundesland
for column in merged.columns:
  if (column == "AnzahlFall" or column=="AnzahlTodesfall") :
      merged[column] = merged[column].replace(r'\s+', np.nan, regex=True)
      merged[column] = merged[column].fillna(0)
  elif (column in bundeslaender):
      merged[column]= column

In [ ]:
#Select relevant dates
today = "22.10.2020"
merged = merged.loc["20.03.2020  00:00:00":today]
merged.to_excel("merged.xlsx")

In [ ]:
# Help functions to select data for diagrams

cases = 1
deaths = 2
freeBeds = 3
occBeds = 4
reserved = 5
bundeslaender = ["Schleswig-Holstein","Hamburg","Niedersachsen","Bremen","Nordrhein-Westfalen","Hessen","Rheinland-Pfalz","Baden-Württemberg","Bayern","Saarland","Berlin","Brandenburg","Mecklenburg-Vorpommern","Sachsen","Sachsen-Anhalt","Thüringen"]

# Cases and deaths overall between 20.03.2020 and today (22.10.2020)

def sumCalcOverall(statistic): # where statistic is either cases, deaths, freeBeds, occBeds or reserved 
  sum = 0
  for bundesland in bundeslaender:
    ind = merged.columns.get_loc(bundesland)
    loc = ind + statistic
    sum = sum + merged.iloc[:,loc].sum()
  return sum

casesGermany = sumCalcOverall(cases)
deathsGermany = sumCalcOverall(deaths)

KeyError: ignored

In [ ]:
# Cases and deaths per bundesland between 20.03.2020 and today (22.10.2020)

def sumCalcBL(bundesland,statistic): # where statistic is either cases, deaths, freeBeds, occBeds or reserved ; bundesland is either "Schleswig-Holstein","Hamburg","Niedersachsen","Bremen","Nordrhein-Westfalen","Hessen","Rheinland-Pfalz","Baden-Württemberg","Bayern","Saarland","Berlin","Brandenburg","Mecklenburg-Vorpommern","Sachsen","Sachsen-Anhalt","Thüringen"
  ind = merged.columns.get_loc(bundesland)
  loc = ind + statistic
  return merged.iloc[:,loc].sum()
 
casesSchleswig = sumCalcBL(bundeslaender[0],cases)
deathsSchleswig = sumCalcBL(bundeslaender[0],deaths)
casesHamburg = sumCalcBL(bundeslaender[1],cases)
deathsHamburg = sumCalcBL(bundeslaender[1],deaths)
casesNiedersachsen = sumCalcBL(bundeslaender[2],cases)
deathsNiedersachsen = sumCalcBL(bundeslaender[2],deaths)
casesBremen = sumCalcBL(bundeslaender[3],cases)
deathsBremen =sumCalcBL(bundeslaender[3],deaths)
casesNordrhein = sumCalcBL(bundeslaender[4],cases)
deathsNordrhein =sumCalcBL(bundeslaender[4],deaths)
casesHessen = sumCalcBL(bundeslaender[5],cases)
deathsHessen = sumCalcBL(bundeslaender[5],deaths)
casesRheinland = sumCalcBL(bundeslaender[6],cases)
deathsRheinland = sumCalcBL(bundeslaender[6],deaths)
casesBaden = sumCalcBL(bundeslaender[7],cases)
deathsBaden = sumCalcBL(bundeslaender[7],deaths)
casesBayern = sumCalcBL(bundeslaender[8],cases)
deathsBayern = sumCalcBL(bundeslaender[8],deaths)
casesSaarland = sumCalcBL(bundeslaender[9],cases)
deathsSaarland = sumCalcBL(bundeslaender[9],deaths)
casesBerlin = sumCalcBL(bundeslaender[10],cases)
deathsBerlin = sumCalcBL(bundeslaender[10],deaths)
casesBrandenburg = sumCalcBL(bundeslaender[11],cases)
deathsBrandenburg = sumCalcBL(bundeslaender[11],deaths)
casesMecklenburg = sumCalcBL(bundeslaender[12],cases)
deathsMecklenburg = sumCalcBL(bundeslaender[12],deaths)
casesSachsen = sumCalcBL(bundeslaender[13],cases)
deathsSachsen = sumCalcBL(bundeslaender[13],deaths)
casesAnhalt = sumCalcBL(bundeslaender[14],cases)
deathsAnhalt = sumCalcBL(bundeslaender[14],deaths)
casesThüringen = sumCalcBL(bundeslaender[15],cases)
deathsThüringen = sumCalcBL(bundeslaender[15],deaths)

KeyError: ignored

In [ ]:
# Select all cases and deaths in Germany based on a certain day
def sumCalcDate(date, statistic): # where statistic is either cases, deaths, freeBeds, occBeds or reserved ; date format Y-M-D
  sum = 0
  selection = merged.loc[date:date]
  for bundesland in bundeslaender:
    ind = selection.columns.get_loc(bundesland)
    loc = ind + statistic
    sum = sum + selection.iloc[:,loc].sum()
  return sum

print (sumCalcDate("20.10.2020",deaths))

KeyError: ignored

In [ ]:
# Select all cases and deaths per bundesland based on a certain day
def sumCalcBundeslandDate(date,bundesland,statistic): # where statistic is either cases, deaths, freeBeds, occBeds or reserved ; date format Y-M-D
  sum = 0
  selection = merged.loc[date:date]
  ind = selection.columns.get_loc(bundesland)
  loc = ind + statistic
  return selection.iloc[:,loc].sum()

print (sumCalcBundeslandDate("19.10.2020","Schleswig-Holstein",deaths))

# 3º and last Step

Finally, the creation of graphs / diagrams for better perception and visualization of the data that will lead us to a conclusion.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Cases, Deaths, Deathrate for entire Germany

casesGermany = sumCalcOverallDate(cases,"2020-03-20","2020-10-22")
deathsGermany = sumCalcOverallDate(deaths,"2020-03-20","2020-10-22")
print(deathsGermany/casesGermany)

barlabel = "German"
value = casesGermany

fig = plt.figure(figsize=(2,5))

plt.bar (barlabel, value, color="black")
plt.ylabel("Number of Infections")
plt.xlabel("Citziens")

plt.show()

NameError: ignored